In [ ]:
#| default_exp utils

# Utilities
> Various utilities

In [ ]:
#| export
from netCDF4 import Dataset
from fastcore.test import test_eq
import pandas as pd
import numpy as np
from typing import Dict
import tomli_w
import tomli
from shapely import MultiPoint, Polygon
import nbformat

## Validation

In [ ]:
#| export
def has_valid_varname(
    var_names:list, # variable (nuclide) names
    cdl_path:str, # Path to MARIS CDL file (point of truth)
):
    "Check that proposed variable names are in MARIS CDL"
    has_valid = True
    with Dataset(cdl_path) as nc:
        grp = nc.groups[list(nc.groups.keys())[0]] # get any group
        for name in var_names:
            if name not in grp.variables.keys():
                has_valid = False
                print(f'"{name}" variable name not found in MARIS CDL')
    
    return has_valid

In [ ]:
VARNAMES = ['lat', 'lon']
test_eq(has_valid_varname(VARNAMES, './files/nc/maris-cdl.nc'), True)

In [ ]:
VARNAMES = ['ba140_invalid', 'ba140_dl']
test_eq(has_valid_varname(VARNAMES, './files/nc/maris-cdl.nc'), False)

"ba140_invalid" variable name not found in MARIS CDL


## Readers & writers

In [ ]:
#| export
def write_toml(fname, cfg):
    print(f'Creating {fname}')
    with open(fname, "wb") as f:
        tomli_w.dump(cfg, f)

In [ ]:
#| export
def read_toml(fname):
    with open(fname, "rb") as f:
        config = tomli.load(f)
    return config

## Geoprocessing

In [ ]:
#| export
def get_bbox(df,
             coord_cols=('lon', 'lat')
            ):
    arr = []
    x, y = coord_cols
    for index, row in df.iterrows():
        arr.append((row[x], row[y]))
    return MultiPoint(arr).envelope

In [ ]:
df = pd.DataFrame({'lon': np.linspace(-10, 5, 20), 'lat':  np.linspace(40, 50, 20)})
bbox = get_bbox(df);

In [ ]:
# To get `lon_min`, `lon_max`, `lat_min`, `lat_max`
bbox.bounds

(-10.0, 40.0, 5.0, 50.0)

In [ ]:
# And its Well-Know Text representation
bbox.wkt

'POLYGON ((-10 40, 5 40, 5 50, -10 50, -10 40))'

## Notebooks

In [ ]:
#| export
def parametrize(notebook:str, # Notebook path
               ):
    "Add `parameters` notebook cell tag when seeing `#| params` special character"
    nb = nbformat.read(notebook, as_version=4)
    cell = [c for c in nb.cells if '#|params' in c.source.replace(' ', '')][0]
    cell.metadata = {'tags': ['parameters']}
    nbformat.write(nb, notebook)